In [1]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  \
pinecone-client \
pandas \
langchain \
sentence-transformers

  Using cached pinecone_client-3.0.2-py3-none-any.whl.metadata (12 kB)
  Using cached pandas-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached langchain-0.1.4-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-2.3.1-py3-none-any.whl.metadata (11 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2023.4-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached SQLAlchemy-2.0.25-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.16-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.1.17-py3-none-any.whl.m

In [15]:
from pinecone import Pinecone
import os
import pandas as pd
import numpy as np

#from sentence_transformers import SentenceTransformer
from langchain.document_loaders import TextLoader
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

# Prepare Data - New Articles

In [17]:
# load multiple document and process documants
loader = DirectoryLoader("./bbc/sport", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [18]:
# split the text into smaler chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_spliter.split_documents(documents)

In [19]:
len(texts)

1505

# Create Embeddings and Vectors

In [21]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
os.environ["CURL_CA_BUNDLE"] = ""

In [22]:
# average_word_embeddings_komninos
# dimension = 300
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HUGGINGFACE_API_KEY")

In [23]:
import requests

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

In [29]:
def model(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}}, verify=False)
    return response.json()

In [30]:
# Must be a list
news_text = []
for text in texts:
    news_text.append(text.page_content)
news_text[0]

'Fuming Robinson blasts officials\n\nEngland coach Andy Robinson insisted he was "livid" after his side were denied two tries in Sunday\'s 19-13 Six Nations loss to Ireland in Dublin.\n\nMark Cueto\'s first-half effort was ruled out for offside before the referee spurned TV replays when England crashed over in the dying minutes. "[I\'m] absolutely spitting. I\'m livid. There\'s two tries we\'ve been cost," Robinson told BBC Sport. "We\'ve got to go back to technology. I don\'t know why we didn\'t." South African referee Jonathan Kaplan ruled that Cueto was ahead of Charlie Hodgson when the fly-half hoisted his cross-field kick for the Sale wing to gather.'

In [31]:
# model = SentenceTransformer('average_word_embeddings_komninos')
# encoded_news = model(news_text, show_progress_bar=True)
encoded_news = model(news_text)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-inference.huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [32]:
len(encoded_news[0])

384

# Setupup Pinecone index
	- Make sure the dimenionality of the embeddings matches the dimensionality of the index

In [33]:
pc.list_indexes()

MaxRetryError: HTTPSConnectionPool(host='api.pinecone.io', port=443): Max retries exceeded with url: /indexes (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [ ]:
pc.delete_index('prices')

In [ ]:
pc.list_indexes()

{'indexes': []}

In [ ]:
from pinecone import PodSpec
pc.create_index(
	'news', 
	dimension=300 ,
	metric='cosine',
	spec=PodSpec(
        	environment="gcp-starter",
        	pod_type="starter",
        	pods=1,
        	replicas=1,
        	shards=1
	)
)

# Upsert vector data

# Query vector data

# Query with metadata filters

# Cleanup